#### Inseriamo ufficio ( tramite id )
#### Inseriamo ente ( tramite id ) numero, anno , ricerca_iqera 
#### per ora entriamo solo nella pagina non facciamo lo scraper dei risultati

In [25]:
import requests
from bs4 import BeautifulSoup

In [26]:
def scraper(ufficio,ente,anno,numero,natura):
    """
    ufficio è l'id che si trova nel file uffici_finanziari (in code)
    ente è l'id da ricavare da enti.json   (esempio 001 )
    numero da file input (rg / decreto / etc )
    anno anno con 4 cifre da file input
    natura  DI  AV  /  EM  OR
    """


    session = requests.Session()
    initial_url = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
    session.get(initial_url) 

    url1 = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
    params1 = {
        'action': 'scegliufficio',
        'ufficio': ufficio,
    }
    headers = {
        'Connection': 'keep-alive',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept': '*/*',
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    body1 = {
        'ufficio': ufficio,
    }
    response1 = session.post(url1, headers=headers, params=params1, data=body1)
    
    if response1:
        url2 = 'https://www1.agenziaentrate.gov.it/servizi/tassazioneattigiudiziari/registrazione.htm'
        params2 = {
            'action': 'scegliente',
        }
        body2 = {
            'ente': ente,
            'anno': anno,
            'numero': numero,
            'natura' : natura,
            'snumero' : ""
        }
        response2 = session.post(url2, headers=headers, params=params2, data=body2)
        if response2 :
            soup2 = BeautifulSoup(response2.text, 'html.parser')
            print('BEAUTIFUL SOUP')
            if 'non trovato' in response2.text:
                #print('riprova')
                return 'non trovato'
            else :
                print('fare il codice per lo scraper dei risultati')
        else :
            print('Qualcosa è andato storto')    

    else:
        print('ufficio non trovato')    

In [27]:
scraper(ufficio='TQD',ente='002',anno='2017',numero='3',natura='DI')

BEAUTIFUL SOUP
fare il codice per lo scraper dei risultati


#### cambia la natura dell'ente in base al tipo diu ricerca ( se monitoria o ppt)

In [28]:
list_natura_monitoria = ['DI','AV']
list_natura_ppt = ['EM','OR']

tipo = 'monitoria'

def tipo_scraper(tipo,ufficio,ente,anno,numero):
    if tipo=='monitoria':
        for natura in list_natura_monitoria:
            result = scraper(ufficio,ente,anno,numero,natura)
            if result != 'non trovato':
                break
    elif tipo == 'ppt':
        for natura in list_natura_ppt:
            result = scraper(ufficio,ente,anno,numero,natura)
            if result != 'non trovato':
                break

tipo_scraper('monitoria',ufficio='TQD',ente='002',anno='2017',numero='3')

BEAUTIFUL SOUP
fare il codice per lo scraper dei risultati


#### sistemiamo l'input prima di mandare lo scaper 

In [120]:
from Pulizia import clean , split_and_expand
import pandas as pd
import json

In [121]:
uffici = pd.read_excel('Resources/uffici_finanziari.xlsx')
uffici = uffici.iloc[1:]
uffici['office'] = uffici['office'].str.lower()
uffici_ut = uffici[uffici['office'].str.contains('ut')]

In [122]:
input = pd.read_excel('Input/Raw/Ricerca F24.xlsx')
input = clean(input)
input['tipo_ricerca'] = input['RICERCA IQERA'].str.lower()
input = input.drop(columns=['RICERCA IQERA'])

mapping = {'albenga':'savona',
 'chiavenna':'sondrio',
 'cossato':'biella',
 'macomer':'nuoro',
 'isili':'nuoro',
 'chiaromonte': 'lagonegro',
 'orzinuovi' : 'brescia',
 'ovada': 'acquiterme',
 'pavullo nel frignano':'modena',
 'ponte san pietro':'bergamo',
 'romano di lombardia':'bergamo',
 'san benedetto del tronto':'ascoli piceno',
 'teano':'caserta',
 'venezia 2 - mestre' : 'venezia',
 'venosa':'potenza',
 'vimercate':'monza',
 'acerra' : 'casoria'
 }

input['Foro'] = (input['Foro']
                    .str.lower()
                    .replace(mapping, regex=True)
                    .str.strip()
                   )
# input


In [123]:
def find_code_for_foro(df1, df2):
    # Initialize an empty list to store the results
    results = []

    # Iterate over each row in df2
    for index2, row2 in df2.iterrows():
        foro = row2['Foro']
        
        # Find matching rows in df1
        matching_rows = df1[df1['office'].str.contains(foro)]

        # Add matching codes to the results list
        for index1, row1 in matching_rows.iterrows():
            results.append({
                'code': row1['code'],
                'office': row1['office'],
                'Foro': row2['Foro']
            })

    # Convert results to a DataFrame
    result_df = pd.DataFrame(results)
    return result_df
result_df = find_code_for_foro(uffici_ut, input)
result_df = result_df.drop_duplicates()

In [124]:

input = pd.merge(input, result_df, on='Foro', how='left')
input = (input
        .drop_duplicates())
input['Tribunale/Gdp'] = (input['Tribunale/Gdp']
                          .str.lower()
                          .str.strip()
)

In [125]:
input[input['code'].isna()]

,NR_Contratto,Debitore,Tribunale/Gdp,Foro,Cliente,nr.rg,year.rg,nr.decreto,year.decreto,nr.repertorio,year.repertorio,nr.crono,year.crono,nr.rgeppt,year.rgeppt,nr.repppt,year.repppt,tipo_ricerca,code,office


#### sopra abbiamo aggiunto il codice dell'ufficio

#### ora dobbiamo aggiungere i codici degli enti

In [126]:
with open('Resources/enti_completi_ut.json', 'r') as file:
    enti_ut = json.load(file)

enti_decoded = {}

for key, values in enti_ut.items():
    tribunale = [item.split(" - ")[0] for item in values if "TRIB" in item.upper()]
    giudice = [item.split(" - ")[0] for item in values if "GIUD" in item.upper()]
    enti_decoded[key] = {"tribunale": tribunale, "giudice": giudice}
# enti_decoded    

In [127]:
enti_decoded['siracusa  ut dpsr']['tribunale']

['001', '007', '009', '012']

In [129]:
def get_codes(row):
    office = row['office']
    type_ = row['Tribunale/Gdp']
    if office in enti_decoded and type_ in enti_decoded[office]:
        return enti_decoded[office][type_]
    else:
        return []

# Apply the function to each row and create a new column 'codes'
input['code_ente'] = input.apply(get_codes, axis=1)

In [ ]:
# fare il mapping dell'ente 
# trovare il modo di fare i 4 / 2 numeri per tipo 
# tipo_scraper(tipo,ufficio,ente,anno,numero)
results =( input
    .apply(lambda row: tipo_scraper(tipo= row['tipo_ricerca'],ufficio= row['code'], ente= 'oo2', anno=row['year.rg'],numero= row['nr.rg']), axis=1)
)